<a href="https://colab.research.google.com/github/acrobyte007/Image_Classification/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile

# Path to the ZIP file
zip_file_path = '/content/EuroSAT.zip'
extraction_path = '/content/EuroSAT_extracted'

# Check if the ZIP file exists
if os.path.exists(zip_file_path):

    os.makedirs(extraction_path, exist_ok=True)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

    # Now list all folders in the extracted directory
    folders = [f for f in os.listdir(extraction_path) if os.path.isdir(os.path.join(extraction_path, f))]

    # Print the names of the folders and their contents
    for folder in folders:
        print(f'Contents of {folder}:')
        folder_path = os.path.join(extraction_path, folder)
        contents = os.listdir(folder_path)
        print(contents)
        print()
else:
    print("The specified ZIP file does not exist.")


Contents of EuroSAT:
['Forest', 'PermanentCrop', 'validation.csv', 'Residential', 'test.csv', 'train.csv', 'label_map.json', 'AnnualCrop', 'Industrial', 'SeaLake', 'River', 'Pasture', 'HerbaceousVegetation', 'Highway']



In [2]:
import pandas as pd

In [3]:
train=pd.read_csv("/content/EuroSAT_extracted/EuroSAT/train.csv")

In [4]:
test=pd.read_csv("/content/EuroSAT_extracted/EuroSAT/test.csv")

In [5]:
validation=pd.read_csv("/content/EuroSAT_extracted/EuroSAT/validation.csv")

In [6]:
train

,Unnamed: 0,Filename,Label,ClassName
0,16257,AnnualCrop/AnnualCrop_142.jpg,0,AnnualCrop
1,3297,HerbaceousVegetation/HerbaceousVegetation_2835...,2,HerbaceousVegetation
2,17881,PermanentCrop/PermanentCrop_1073.jpg,6,PermanentCrop
3,2223,Industrial/Industrial_453.jpg,4,Industrial
4,4887,HerbaceousVegetation/HerbaceousVegetation_1810...,2,HerbaceousVegetation
...,...,...,...,...
18895,4498,HerbaceousVegetation/HerbaceousVegetation_1952...,2,HerbaceousVegetation
18896,1149,Pasture/Pasture_1252.jpg,5,Pasture
18897,15489,AnnualCrop/AnnualCrop_2332.jpg,0,AnnualCrop
18898,6287,Residential/Residential_332.jpg,7,Residential


In [7]:
import pandas as pd
from pathlib import Path
from PIL import Image

In [9]:
parent_dir=Path("/content/EuroSAT_extracted/EuroSAT")

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [11]:
# Convert parent_dir to a string and concatenate with the Filename
train['image_path'] = train['Filename'].apply(lambda x: str(parent_dir) + "/" + x)
validation['image_path'] = validation['Filename'].apply(lambda x: str(parent_dir) + "/" + x)


In [12]:

# Prepare ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1.0/255)

# Function to load images from DataFrame
def load_data_from_dataframe(df, datagen, target_size=(224, 224), batch_size=32, shuffle=True):
    return datagen.flow_from_dataframe(
        dataframe=df,
        x_col='image_path',
        y_col='ClassName',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=shuffle
    )

# Create the training and validation data generators
train_generator = load_data_from_dataframe(train, train_datagen)
val_generator = load_data_from_dataframe(validation, val_datagen, shuffle=False)

Found 18900 validated image filenames belonging to 10 classes.
Found 5400 validated image filenames belonging to 10 classes.


In [13]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load the VGG16 model with pre-trained weights, excluding the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [14]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10  # Adjust epochs as needed
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


590/590 ━━━━━━━━━━━━━━━━━━━━ 298s 469ms/step - accuracy: 0.4278 - loss: 1.7731 - val_accuracy: 0.6929 - val_loss: 0.8720
Epoch 2/10
  1/590 ━━━━━━━━━━━━━━━━━━━━ 1:17 131ms/step - accuracy: 0.5625 - loss: 1.1489

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


590/590 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.5625 - loss: 1.1489 - val_accuracy: 0.7500 - val_loss: 0.7987
Epoch 3/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 277s 443ms/step - accuracy: 0.5906 - loss: 1.1355 - val_accuracy: 0.7379 - val_loss: 0.7725
Epoch 4/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step - accuracy: 0.5938 - loss: 0.9634 - val_accuracy: 0.8333 - val_loss: 0.5613
Epoch 5/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 263s 441ms/step - accuracy: 0.6192 - loss: 1.0528 - val_accuracy: 0.7759 - val_loss: 0.6465
Epoch 6/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step - accuracy: 0.8125 - loss: 0.6250 - val_accuracy: 0.8750 - val_loss: 0.4103
Epoch 7/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 261s 438ms/step - accuracy: 0.6433 - loss: 0.9819 - val_accuracy: 0.7610 - val_loss: 0.7137
Epoch 8/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.5000 - loss: 1.2713 - val_accuracy: 0.7917 - val_loss: 0.5714
Epoch 9/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 260s 438ms/step - accuracy: 0.6374 - loss: 0.9825 - val

In [15]:
# Evaluate on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


169/169 ━━━━━━━━━━━━━━━━━━━━ 23s 134ms/step - accuracy: 0.7829 - loss: 0.5988
Validation Loss: 0.5992153882980347
Validation Accuracy: 0.787407398223877


In [18]:
# Define the test CSV path
test_csv = "/content/EuroSAT_extracted/EuroSAT/test.csv"

# Load the test CSV file
test_df = pd.read_csv(test_csv)

# Add a new column for the full image path
test_df['image_path'] = test_df['Filename'].apply(lambda x: str(parent_dir) + "/" + x)

# Prepare the test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Create the test data generator
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='ClassName',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 2700 validated image filenames belonging to 10 classes.


In [17]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


85/85 ━━━━━━━━━━━━━━━━━━━━ 18s 212ms/step - accuracy: 0.8016 - loss: 0.5726
Test Loss: 0.5806007385253906
Test Accuracy: 0.7985185384750366


In [19]:
model.save('EuroSAT_model.h5')